Imports - импортируем все необходимое

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [43]:
X_full = pd.read_csv('db.csv', index_col='id')
X_test_full = pd.read_csv('db.csv', index_col='id')

In [44]:
X_full

,giving_dttm,enrollment_dttm,down_time_train,facts_departures_id,transportation_sheet_id,interval_giving,departure_dttm,end_pending_dttm,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time
id,,,,,,,,,,,,,
24,2021-10-05 04:01:00,2021-10-05 04:36:00,0.583333,34,920,0.0,2021-10-05 11:37:00,2021-10-05 06:40:00,4.950000,-7.600000,2.40000,-2.050008,NaN
32,2021-10-01 07:10:00,2021-10-01 09:10:10,2.002778,14,778,0.0,2021-10-02 00:50:00,2021-09-30 22:50:00,26.000000,-17.666667,NaN,NaN,NaN
33,2021-10-01 05:00:00,2021-10-01 07:40:00,2.666667,22,763,0.0,2021-10-01 02:02:00,2021-09-30 23:45:00,2.283333,2.966667,NaN,NaN,NaN
34,2021-10-06 17:30:00,2021-10-06 19:30:00,2.000000,43,939,0.0,2021-10-06 14:02:00,2021-10-06 07:30:00,6.533333,3.466667,NaN,0.900000,NaN
35,2021-10-06 19:20:00,2021-10-06 22:20:00,3.000000,44,941,0.0,2021-10-06 16:09:00,2021-10-06 12:30:00,3.650000,3.183333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5394,2022-04-30 17:20:00,2022-04-30 18:05:00,0.750000,5372,16838,0.0,2022-04-30 14:00:00,2022-04-30 12:30:00,1.500000,3.333333,NaN,NaN,0.433333
5395,2022-04-30 18:05:00,2022-04-30 20:30:00,2.416667,5371,16839,0.0,2022-04-30 15:24:00,2022-04-30 09:40:00,5.733333,2.683333,NaN,NaN,NaN
5396,2022-04-30 20:45:00,2022-04-30 21:15:00,0.500000,5344,16749,0.0,2022-04-29 08:51:00,2022-04-29 06:30:00,2.350000,35.900000,5.15004,1.116672,4.266667


Data explorationData exploration

In [45]:
print(X_full.columns)

Index(['giving_dttm', 'enrollment_dttm', 'down_time_train',
       'facts_departures_id', 'transportation_sheet_id', 'interval_giving',
       'departure_dttm', 'end_pending_dttm', 'idle_train', 'travel_time',
       'crew_down_time', 'crew_down_time_plan', 'intermediate_down_time'],
      dtype='object')


In [46]:
X_full.sample(5)

,giving_dttm,enrollment_dttm,down_time_train,facts_departures_id,transportation_sheet_id,interval_giving,departure_dttm,end_pending_dttm,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time
id,,,,,,,,,,,,,
2306,2021-12-29 18:30:00,2021-12-29 20:30:00,2.000000,2259,6841,0.0,2021-12-29 15:05:00,2021-12-29 10:40:00,4.416667,3.416667,NaN,NaN,NaN
15599,2023-06-08 17:30:00,2023-06-08 19:45:00,2.250000,15582,56707,0.0,2023-06-08 13:42:00,2023-06-08 00:30:00,13.200000,3.800000,NaN,NaN,0.150000
28943,2024-11-13 20:02:00,2024-11-13 21:05:00,1.050000,29047,109513,0.0,2024-11-13 16:40:00,2024-11-13 13:30:00,3.166667,3.366667,NaN,NaN,NaN
32258,2025-03-23 13:06:00,2025-03-23 13:40:00,0.566667,32383,122212,0.0,2025-03-23 09:33:00,2025-03-23 06:00:00,3.550000,3.550000,NaN,NaN,0.200000
25910,2024-07-22 18:16:00,2024-07-22 18:16:00,0.000000,25988,97823,0.0,2024-07-22 14:02:00,2024-07-22 09:30:00,4.533333,4.233333,NaN,NaN,0.733333


In [47]:
X_full.describe(include='object')

,giving_dttm,enrollment_dttm,departure_dttm,end_pending_dttm
count,33302,33302,33302,33302
unique,32803,32187,32907,32049
top,2024-07-24 17:42:00,2021-10-03 13:10:00,2022-06-29 01:35:00,2021-10-24 16:50:00
freq,3,4,3,3


In [48]:
X_full.drop(['facts_departures_id'], axis=1, inplace=True)
X_full.drop(['transportation_sheet_id'], axis=1, inplace=True)
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33302 entries, 24 to 5969
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   giving_dttm             33302 non-null  object 
 1   enrollment_dttm         33302 non-null  object 
 2   down_time_train         33302 non-null  float64
 3   interval_giving         33302 non-null  float64
 4   departure_dttm          33302 non-null  object 
 5   end_pending_dttm        33302 non-null  object 
 6   idle_train              33302 non-null  float64
 7   travel_time             33302 non-null  float64
 8   crew_down_time          11797 non-null  float64
 9   crew_down_time_plan     3866 non-null   float64
 10  intermediate_down_time  19889 non-null  float64
dtypes: float64(7), object(4)
memory usage: 3.0+ MB


Data preparation - осуществляем базовые преобразования датасета

In [49]:
# Удаляем из датасета строки, в которых недостаточно данных для обучения.
X_full.dropna(axis=0, subset=['idle_train'], inplace=True)
# Разбиваем датасет на цель (y) и входные данные (X)
y = X_full.idle_train
X_full.drop(['idle_train'], axis=1, inplace=True)

# Разбиваем датасет на train/val
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [50]:
# В наших данных есть нечисловые значения. Они затрудняют обработку
# поэтому мы оставим только те данные, где нечисловые значения имеют меньше 10 разных значений
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]

# и все числовые данные
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]

# все столбцы, в которых количество пропусков больше 10% X[cname].isnull().sum()

# оставим в датасете только те колонки, с которыми хотим работать дальше
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

Sanity check

In [51]:
print(len(X_train.columns), len(X_train_full.columns))
X_train.head()

6 10


,down_time_train,interval_giving,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time
id,,,,,,
10672,0.583333,0.0,26.333333,3.866664,NaN,3.166667
8286,2.000000,0.0,2.750000,NaN,NaN,NaN
7247,2.000000,0.0,2.700000,NaN,NaN,NaN
30638,0.500000,0.0,3.066667,NaN,NaN,NaN
11036,1.916667,0.0,-20.400000,NaN,NaN,0.683333


#Pipeline

Baseline - базовый пайплайн, от которого мы будем отталкиваться в дальнейшем

###Data preparation (preprocessing)

In [52]:
# Инициализируем препроцессор для заполнения недостающих числовых значений
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Инициализируем препроцессор для заполнения недостающих НЕ числовых значений
# Добавляем шаг кодирования НЕ числовых значений в числовые
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Создаем общий препроцессор данных, соединив первые два
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ])

### Model

In [53]:
# Инициализируем модель
model = RandomForestRegressor(n_estimators=100, random_state=0, verbose=True)

In [54]:
# Создаем пайплайн из модели и препроцессора
classifier = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)],
                      verbose=True)

### Training

In [55]:
# Передаем обучающие данные в пайплайн, обучаем
classifier.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    3.7s


[Pipeline] ............. (step 2 of 2) Processing model, total=   7.1s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.0s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['down_time_train',
                                                   'interval_giving',
                                                   'travel_time',
                                                   'crew_down_time',
                                                   'crew_down_time_plan',
                                                   'intermediate_down_time']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', RandomForestRegressor(random_state=0, verbose=True))],
         verbose=True)

### Model evaluation

In [56]:
# Передаем валидационные данные в пайплайн, получаем предсказания
preds = classifier.predict(X_valid)

# Оцениваем точность модели
print('MAE:', mean_absolute_error(y_valid, preds))

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


MAE: 2.213635454676557


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [57]:
y_valid.mean()

np.float64(3.9179703169193814)

In [63]:
(1 - 2.213635454676557/3.9179703169193814)*100

43.500453663031

In [59]:
print('MSE:', mean_squared_error(y_valid, preds))

MSE: 11.021351808144292


In [64]:
routes_to_check = 80
print(X_valid.iloc[routes_to_check].to_string())
print()
print(y_valid.iloc[routes_to_check])

down_time_train            1.266667
interval_giving            1.000000
travel_time               34.883333
crew_down_time             9.133344
crew_down_time_plan             NaN
intermediate_down_time     1.933333

0.433333


In [61]:
classifier.predict(X_valid.iloc[routes_to_check:routes_to_check + 1])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([2.32383338])

In [62]:
r2_score(y_valid, preds)

-0.12168746593635227